In [1]:
!pip install --upgrade category_encoders matplotlib pandas_profiling seaborn psutil xlrd eli5 pdp pdpbox shap
import sys, getpass, pandas_profiling
a = getpass.getuser()
### examples appending of folder
# A
sys.path.append('/home/{}/Code/Github-Repos/BFDS_Library/'.format(a))
# B
#sys.path.append('/Users/{}/Programs/BFDS_Library/'.format(a))

# Pull in library if in Google Colab
if 'google.colab' in sys.modules:
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/jeibloo/BFDS_Library.git
    !git pull origin master

#from BFDS_Library import ACompass, BWarp, CToolbox 
import AMap as am
import BCook as bc
import CTools as ct

Requirement already up-to-date: category_encoders in /home/ygeun/.local/share/virtualenvs/coral-death-Jt6ndagv/lib/python3.7/site-packages (2.0.0)
Requirement already up-to-date: matplotlib in /home/ygeun/.local/share/virtualenvs/coral-death-Jt6ndagv/lib/python3.7/site-packages (3.1.1)
Requirement already up-to-date: pandas_profiling in /home/ygeun/.local/share/virtualenvs/coral-death-Jt6ndagv/lib/python3.7/site-packages (2.3.0)
Requirement already up-to-date: seaborn in /home/ygeun/.local/share/virtualenvs/coral-death-Jt6ndagv/lib/python3.7/site-packages (0.9.0)
Requirement already up-to-date: psutil in /home/ygeun/.local/share/virtualenvs/coral-death-Jt6ndagv/lib/python3.7/site-packages (5.6.3)
Requirement already up-to-date: xlrd in /home/ygeun/.local/share/virtualenvs/coral-death-Jt6ndagv/lib/python3.7/site-packages (1.2.0)
Requirement already up-to-date: eli5 in /home/ygeun/.local/share/virtualenvs/coral-death-Jt6ndagv/lib/python3.7/site-packages (0.10.0)
Requirement already up-to

Cool dataset from [Reefbase](http://www.reefbase.org/gis_maps/useofmaps.aspx).

In [2]:
import pandas as pd
pd.options.display.float_format = None
df = pd.read_excel('./dataset/CoralBleaching.xlsm')

In [3]:
target = 'BLEACHING_SEVERITY'

In [4]:
def wgle(df):
    # Group by bleaching or not
    df[target] = df[target].replace({
        'Low':2, 'HIGH':4, 'Medium':3, 'Severity Unknown':1,
        'No Bleaching':0})

    # Get all features for ease-of-use
    features, num_feats, cat_feats = bc.pantryFeatures(df)
    
    # Drop columns with almost no values.
    df = bc.diceUglyCols(df, features, percent=50)
    
    # Reduce cardinality of a few columns
    df['LOCATION'].replace({'No Name':'other'},inplace=True)
    df = bc.simmerCardinality(df, 'LOCATION', amt=400)
    df = bc.simmerCardinality(df, 'SOURCE', amt=400)
    
    # Drop severity code cause that's obviously a leakage
    df = df.drop(['SEVERITY_CODE','ID'],axis=1)
    
    # Special weird values
    df['LAT'] = df['LAT'].replace({-10269:-10.269})
    
    return df

In [5]:
#df.profile_report()

In [6]:
dfw = wgle(df)
dfw = ct.downcastDataFrame(dfw)

Mem. usage decreased to  0.34 Mb (35.2% reduction)


In [7]:
'''### Visualizations
import seaborn as sns
import matplotlib.pyplot as plt

fig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(20,10))
ax1.title.set_text('Latitude\nNorth\n^\n|\nv\nSouth')
ax2.title.set_text('Longitude\nWest <-> East')
sns.distplot(dfw['LAT'], ax=ax1, color='y',vertical=True)
sns.distplot(dfw['LON'], ax=ax2, color='b',vertical=False)'''

"### Visualizations\nimport seaborn as sns\nimport matplotlib.pyplot as plt\n\nfig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(20,10))\nax1.title.set_text('Latitude\nNorth\n^\n|\nv\nSouth')\nax2.title.set_text('Longitude\nWest <-> East')\nsns.distplot(dfw['LAT'], ax=ax1, color='y',vertical=True)\nsns.distplot(dfw['LON'], ax=ax2, color='b',vertical=False)"

In [8]:
dfw.nunique()

REGION                   6
SUBREGION               17
COUNTRY                112
LOCATION               400
LAT                   1881
LON                    942
MONTH                   12
YEAR                    39
BLEACHING_SEVERITY       5
SOURCE                 401
COUNTRY_CODE           112
dtype: int64

In [9]:
pd.value_counts(dfw['LOCATION'].values,sort=True)

other                                                                 3581
U/N Reef                                                               188
nan                                                                    123
Paso Lagrimas, Pez Maya, Reserva de la Biosfera de Sian Ka'an           90
San Miguel de Ruiz, Pez Maya, Reserva de la Biosfera de Sian Ka'an      72
                                                                      ... 
Waitabu village, Taveuni                                                 2
Petricola Shoal                                                          2
Myrmidon Rf                                                              2
Dingo Bch, Nelly Bay                                                     2
La Caleta                                                                2
Length: 400, dtype: int64

In [10]:
import numpy as np
### Export wrangled dataframe
dfw.sort_values(by="YEAR",inplace=True)
### Group into decades
df_chunked = dfw.copy()
'''
df_chunked = df_chunked.mask(
    (df_chunked['YEAR'] <= 1969, '60s'),
    (((df_chunked['YEAR'] <= 1979) & (df_chunked['YEAR'] >= 1970)),'70s'),
    (((df_chunked['YEAR'] <= 1989) & (df_chunked['YEAR'] >= 1980)),'80s'),
    (((df_chunked['YEAR'] <= 1999) & (df_chunked['YEAR'] >= 1990)),'90s'),
    (((df_chunked['YEAR'] <= 2012) & (df_chunked['YEAR'] >= 2000)),'00s')
)
'''
df_chunked['YEAR'] = np.where(df_chunked['YEAR'].between(1960,1969),60,df_chunked['YEAR'])
df_chunked['YEAR'] = np.where(df_chunked['YEAR'].between(1970,1979),70,df_chunked['YEAR'])
df_chunked['YEAR'] = np.where(df_chunked['YEAR'].between(1980,1989),80,df_chunked['YEAR'])
df_chunked['YEAR'] = np.where(df_chunked['YEAR'].between(1990,1999),90,df_chunked['YEAR'])
df_chunked['YEAR'] = np.where(df_chunked['YEAR'].between(2000,2012),2000,df_chunked['YEAR'])
df_chunked['YEAR'] = df_chunked['YEAR'].replace({60:'1960s',70:'1970s',80:'1980s',
                                                 90:'1990s',2000:'2000s'})
df_chunked.head()
df_chunked.to_csv('./CoralBleachingClean.csv')

In [11]:
print("Cut down {} columns.".format(df.shape[1]-dfw.shape[1]))

Cut down 17 columns.


In [12]:
# Split into Train Test & Val by year
test_mask = (dfw['YEAR'] > 2005)
vali_mask = ((dfw['YEAR'] > 2003) & (df['YEAR'] <= 2005))
trin_mask = (dfw['YEAR'] <= 2003)
# Use masks
test = dfw[test_mask].copy()
trin = dfw[trin_mask].copy()
vali = dfw[vali_mask].copy()

/home/ygeun/.local/share/virtualenvs/coral-death-Jt6ndagv/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [13]:
print("ASSURANCE CHECK\nTest shape:{}\nTrain shape:{}\nValidation shape:{}".format(
    test.shape,trin.shape,vali.shape)
)

ASSURANCE CHECK
Test shape:(1058, 11)
Train shape:(4167, 11)
Validation shape:(965, 11)


In [14]:
print(bc.soupBaseModel(trin, target, 1),
      bc.soupBaseModel(vali, target, 1),
      bc.soupBaseModel(test, target, 1))

Classification
0    0.244060
4    0.232541
2    0.230622
1    0.150468
3    0.142309
Name: BLEACHING_SEVERITY, dtype: float64
Classification
2    0.339896
3    0.227979
1    0.220725
0    0.121244
4    0.090155
Name: BLEACHING_SEVERITY, dtype: float64
Classification
1    0.341210
0    0.324197
2    0.132325
4    0.109641
3    0.092628
Name: BLEACHING_SEVERITY, dtype: float64
None None None


In [15]:
### Chop the sets up
features, _, _ = bc.pantryFeatures(trin)
X_train = trin[features].drop(target,axis=1)
y_train = trin[target]

X_test = test[features].drop(target,axis=1)
y_test = test[target]

X_vali = vali[features].drop(target,axis=1)
y_vali = vali[target]

print("ASSURANCE CHECK\nTrain cols:\t{}\nValidtn cols:\t{}\nTest cols:\t{}".format(
    len(X_train.columns)+1,
    len(X_test.columns)+1,
    len(X_vali.columns)+1)
)

ASSURANCE CHECK
Train cols:	11
Validtn cols:	11
Test cols:	11


In [16]:
# Majority Class Baseline
bc.soupBaseModel(df, target, 1)

Classification
0    0.238611
2    0.230856
1    0.194023
4    0.189338
3    0.147173
Name: BLEACHING_SEVERITY, dtype: float64


In [17]:
df[target].value_counts()

0    1477
2    1429
1    1201
4    1172
3     911
Name: BLEACHING_SEVERITY, dtype: int64

In [18]:
### Start fitting this classification model
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

In [19]:
pipey = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(
        n_estimators=200,
        class_weight='balanced',
        n_jobs=-1,
        random_state=4
    )
)

In [20]:
from sklearn.metrics import accuracy_score
# Train the model on these two beauts
pipey.fit(X_train, y_train);

In [21]:
# Preserve the exact pipeline with the exact fit
from joblib import dump
dump(pipey, 'pipey.joblib')

['pipey.joblib']

In [22]:
X_train.shape, y_train.shape, X_vali.shape,y_vali.shape

((4167, 10), (4167,), (965, 10), (965,))

In [23]:
# Print out what y_val prediction was
#print("Val Set AUC ROC: {}".format(roc_auc_score(y_vali,y_pred_proba)))
print("Val Set score: {}".format(pipey.score(X_vali,y_vali)))

Val Set score: 0.45699481865284974


In [24]:
# Test cross-validation
from sklearn.model_selection import cross_val_score
cross_val_score(pipey, X_train, y_train, cv=10, scoring='accuracy')

array([0.23627685, 0.50717703, 0.46650718, 0.50119904, 0.39328537,
       0.48441247, 0.49160671, 0.44096386, 0.48674699, 0.51932367])

What in the ... lol 0.61~...a little better than guessing.

In [25]:
X_combo = pd.concat([X_train, X_vali])
y_combo = pd.concat([y_train, y_vali])
# Train the pipeline on the combo
pipey.fit(X_combo, y_combo);
y_pred = pipey.predict(X_test)

In [26]:
print("Test Set score: {}".format(accuracy_score(y_pred,y_test)))

Test Set score: 0.387523629489603


In [27]:
np.where(y_pred == 4)

(array([ 182,  220,  382,  428,  429,  443,  446,  461,  465,  486,  488,
         489,  496,  503,  508,  646,  723,  726,  739,  750,  755,  757,
         760,  767,  768,  777,  786,  790,  791,  805,  810,  819,  824,
         830,  852,  889,  922,  956, 1006]),)

In [28]:
X_combo.REGION.unique()

array(['Americas', 'Asia', 'Australia', 'Pacific', 'Africa',
       'Middle East', nan], dtype=object)

In [29]:
'''from pdpbox.pdp import pdp_plot, pdp_isolate
from pdpbox import pdp

encoder = pipey.named_steps['ordinalencoder']
imputer = pipey.named_steps['simpleimputer']
model = pipey.named_steps['randomforestclassifier']

X_encoded = encoder.transform(X_combo)
X_imputed = imputer.transform(X_encoded)
X_imputed = pd.DataFrame(X_imputed, columns=X_combo.columns)

features, _, _ = bc.pantryFeatures(X_imputed)
feature = 'COUNTRY_CODE'

pdp_dist = pdp.pdp_isolate(
    model=pipey,
    dataset=X_encoded,
    model_features=features,
    feature=feature,
    num_grid_points=100
)
plot_params = {
    # Plot title etc
    'title': 'PDP for feature {}'.format(feature.lower()),
    'subtitle': "In 5 groups for some reason",
    'markersize': 5,
    'fill_color': '#af4f4f'
}
fig, axes = pdp_plot(pdp_dist,plot_params=plot_params, feature_name=feature,
         plot_lines=True, frac_to_plot=50)
''';

In [30]:
'''import shap
rowNum = 134
row = X_combo.iloc[rowNum]
row_processed = X_imputed.iloc[rowNum]

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(row_processed)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value[0],
    shap_values=shap_values[0],
    features=row
)''';